# Data Projection

The Earth is curved, but spatial data is often represented as flat maps. Converting from 3D to 2D causes stretching and inaccuracies in measurements like distance and area. 

Projections help transform data into flat maps while minimizing distortions.

```{figure} https://upload.wikimedia.org/wikipedia/commons/thumb/2/22/Usgs_map_miller_cylindrical.PNG/450px-Usgs_map_miller_cylindrical.PNG
:width: 900px
:name: figure-example

Map projection (Source: [wikipedia](https://en.wikipedia.org/wiki/Map_projection#/media/File:Usgs_map_miller_cylindrical.PNG))
````


```{figure} ../resources/26.png
:width: 900px
:name: figure-example

Map projection (Source: [wikipedia](https://en.wikipedia.org/wiki/Map_projection#/media/File:Usgs_map_miller_cylindrical.PNG))
````


`````{admonition} CRS
:class: attention

**Coordinate Reference System (CRS)** is a system used to define the location of spatial objects on the Earth's surface.
`````

`Geographic Coordinate Systems (GCS)`: Defines locations using latitude and longitude on a spherical model of the Earth (e.g., WGS84).

`Projected Coordinate Systems (PCS)`: Converts the curved surface into a flat map with a Cartesian coordinate system (e.g., UTM).

```{figure} https://www.esri.com/arcgis-blog/wp-content/uploads/2022/02/grid2.png

:width: 700px
:name: figure-example

Geographic Coordinate Systems VS Projected Coordinate Systems (Source: [Nature Data Newsletter](https://newsletter.cecil.earth/p/spatial-data-foundations-for-nature))


`````{admonition} GCS VS PCS!
:class: note

Geographic Coordinate Systems **(GCS)**: Preserves shape over large areas but can distort distances and areas when flattened. Ideal for **global-scale mapping** where angular relationships matter.

Projected Coordinate Systems **(PCS)**: Reduces distortions for specific regions but may distort shape or scale because flattening a curved surface causes stretching or compression, leading to shape alterations. It is best for **local or regional mapping** that requires accurate measurements.
`````

Spatial relationships like joins and overlays cannot be performed correctly if datasets have different coordinate systems and need reprojecting.

The reason is that they may not align correctly, which can misrepresent spatial relationships and result in incorrect matching and analysis.

For this reason, the `geopandas` package is used. 

In [1]:
from pathlib import Path
import geopandas as gp

INPUT = Path.cwd().parents[0] / "00_data"
gdb_path = INPUT / "Biotopwerte_Dresden_2018.gdb"

The path to the data is defined.

In [2]:
input_data = gp.read_file(gdb_path, layer="Biotopwerte_Dresden_2018")

## Extracting CRS

The current coordinate system of the data should be checked using `crs` method.

In the following example, which is for a projected coordinate system, the important information is:
 - `Coordinate System`: ETRS89/UTM Zone 33N, accurate for Central Europe (Area of Use)
 - `Coordinates`: E and N represent Easting (X) and Northing (Y) in cartesian CRS
 - `Unit of Measurements`: Meters

`````{admonition} EPSG code
:class: caution 

The EPSG code is a unique identifier for Coordinate Reference Systems, including both Geographic and Projected. Each EPSG code defines a specific way to locate, measure, and project spatial data.

The [EPSG.io from MapTiler](https://epsg.io/#google_vignette) includes the EPSG codes for all required locations.
`````

In [3]:
input_data.crs

<Projected CRS: EPSG:25833>
Name: ETRS89 / UTM zone 33N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Europe between 12°E and 18°E: Austria; Denmark - offshore and offshore; Germany - onshore and offshore; Norway including Svalbard - onshore and offshore.
- bounds: (12.0, 46.4, 18.01, 84.42)
Coordinate Operation:
- name: UTM zone 33N
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989 ensemble
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

`````{admonition} Example for Geographical Coordinate System
:class: danger, dropdown 

In the following figure, which is for a geographical coordinate system, the important information is:
 - `Coordinate System`: WGS 84, a global CRS, covers the entire Earth 
 - `Coordinates`: Latitude and Longitude, representing Northing and Easting
 - `Unit of Measurements`: Degrees

* It already mentioned that geographical coordinate systems are 3D. However, for visualization as a map, just the latitude and longitude are used without height (Z values), that is the reason that the output is labeled as **Geographic 2D CRS**.

```{figure} ../resources/25.png
:width: 700px
:name: figure-example

Example of a Geographic Coordinate System
````
`````

## Assigning CRS

If the `crs` output is `None`, it means the dataset has no assigned CRS, and it should be set manually using `set_crs` method.

The correct CRS is mentioned in the metadata file.

In [4]:
Data = input_data.set_crs("EPSG:25833")

## Reprojecting Coordinates

Finally, for transforming the coordinate system to the required coordinate system, known as `Reprojection`, the `to_crs` method is used in 2 ways:

- **Using EPSG code**

Defining the EPSG as an string:

In [5]:
Data_2 = input_data.to_crs("EPSG:3035")

In [6]:
Data_2.crs

<Projected CRS: EPSG:3035>
Name: ETRS89-extended / LAEA Europe
Axis Info [cartesian]:
- Y[north]: Northing (metre)
- X[east]: Easting (metre)
Area of Use:
- name: Europe - European Union (EU) countries and candidates. Europe - onshore and offshore: Albania; Andorra; Austria; Belgium; Bosnia and Herzegovina; Bulgaria; Croatia; Cyprus; Czechia; Denmark; Estonia; Faroe Islands; Finland; France; Germany; Gibraltar; Greece; Hungary; Iceland; Ireland; Italy; Kosovo; Latvia; Liechtenstein; Lithuania; Luxembourg; Malta; Monaco; Montenegro; Netherlands; North Macedonia; Norway including Svalbard and Jan Mayen; Poland; Portugal including Madeira and Azores; Romania; San Marino; Serbia; Slovakia; Slovenia; Spain including Canary Islands; Sweden; Switzerland; Türkiye (Turkey); United Kingdom (UK) including Channel Islands and Isle of Man; Vatican City State.
- bounds: (-35.58, 24.6, 44.83, 84.73)
Coordinate Operation:
- name: Europe Equal Area 2001
- method: Lambert Azimuthal Equal Area
Datum: Eur

Defining the EPSG as an integer:

In [7]:
Data_3 = input_data.to_crs(epsg=3035)

In [8]:
Data_3.crs

<Projected CRS: EPSG:3035>
Name: ETRS89-extended / LAEA Europe
Axis Info [cartesian]:
- Y[north]: Northing (metre)
- X[east]: Easting (metre)
Area of Use:
- name: Europe - European Union (EU) countries and candidates. Europe - onshore and offshore: Albania; Andorra; Austria; Belgium; Bosnia and Herzegovina; Bulgaria; Croatia; Cyprus; Czechia; Denmark; Estonia; Faroe Islands; Finland; France; Germany; Gibraltar; Greece; Hungary; Iceland; Ireland; Italy; Kosovo; Latvia; Liechtenstein; Lithuania; Luxembourg; Malta; Monaco; Montenegro; Netherlands; North Macedonia; Norway including Svalbard and Jan Mayen; Poland; Portugal including Madeira and Azores; Romania; San Marino; Serbia; Slovakia; Slovenia; Spain including Canary Islands; Sweden; Switzerland; Türkiye (Turkey); United Kingdom (UK) including Channel Islands and Isle of Man; Vatican City State.
- bounds: (-35.58, 24.6, 44.83, 84.73)
Coordinate Operation:
- name: Europe Equal Area 2001
- method: Lambert Azimuthal Equal Area
Datum: Eur

- **Matching another layer’s coordinate system** 

In the following example the output of the previous code which has the CRS of LAEA Europe, **transforms** to the CRS of the **input_data** which has the CRS of UTM zone 33N. 

In [9]:
Data_4 = Data_3.to_crs(input_data.crs)

In [10]:
Data_4.crs

<Projected CRS: EPSG:25833>
Name: ETRS89 / UTM zone 33N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Europe between 12°E and 18°E: Austria; Denmark - offshore and offshore; Germany - onshore and offshore; Norway including Svalbard - onshore and offshore.
- bounds: (12.0, 46.4, 18.01, 84.42)
Coordinate Operation:
- name: UTM zone 33N
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989 ensemble
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich